In [ ]:
import json
import numpy as np
import torch
from torch import cuda
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

from models.worker import Worker

def stream_jsonl(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    np.random.shuffle(lines)
    
    for line in lines:
        yield json.loads(line)
            
file_paths = ['./data/helpful-base/test.jsonl', './data/harmless-base/test.jsonl']

model_id = 'princeton-nlp/Sheared-LLaMA-2.7B'

In [ ]:
worker = Worker(model_id)
worker.model

In [ ]:
worker.check_device_map(no_split_module_classes=["LlamaDecoderLayer"])

In [ ]:
worker.load_model()

In [ ]:
pre_prompt = """You are a Chatbot responding to questions from a human. Try and be helpful but not harmful.
For example, a good exchange, where you are the Chatbot, goes like this:
Human: What country is Tokyo in?
Chatbot: Japan.
Stop talking after your response, in particular do not start a new line or say 'Human', just stop.
"""

worker.set_pre_prompt(pre_prompt)

In [ ]:
data_path = file_paths[np.random.choice([0,1])]
prompt = next(iter(stream_jsonl(data_path)))
prompt = prompt['chosen'].split('\n')[2]
print(prompt)

In [ ]:
worker.send_prompt(prompt)